**Data and Preparation **
*  Step 1 - Load libraries
*  Step 2 - Download the dataset
*  Step 3 - Normalize data
*  Step 4 - Create CV-dataset


In [ ]:
# Import libraries
import numpy as np
import pandas as pd
import torch
from torchvision import datasets,transforms
from torch import nn, optim 
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

# Download the dataset
train_data = pd.read_csv('../input/train.csv')

# Normalize the dataset
targets_np = train_data.label.values
variables_np = train_data.loc[:,train_data.columns != "label"].values/255

# Create CV-dataset
variables_train, variables_test, targets_train, targets_test = train_test_split(variables_np, targets_np, test_size = 0.2, random_state = 5)

**Model **
* Create tensors
* Plot number
* Create classifier

> Pytorch: https://pytorch.org/docs/stable/index.html



In [ ]:
# Create tensors
variablesTrain = torch.FloatTensor(variables_train)
targetsTrain = torch.LongTensor(targets_train)
variablesTest = torch.FloatTensor(variables_test)
targetsTest = torch.LongTensor(targets_test)
print(type(variablesTest))
print(type(targetsTest))
trainset = torch.utils.data.TensorDataset(variablesTrain,targetsTrain)
testset = torch.utils.data.TensorDataset(variablesTest,targetsTest)
trainloader = torch.utils.data.DataLoader(trainset, batch_size = 64, shuffle = False)
testloader = torch.utils.data.DataLoader(testset, batch_size = 64, shuffle = False)

# Plot number
plt.imshow(variables_np[15].reshape(28,28))
plt.axis("off")
plt.title(str(targets_np[15]))
plt.savefig('graph.png')
plt.show()

# Create classifier
class classifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1=nn.Linear(784, 256)
        self.fc2=nn.Linear(256,128)
        self.fc3=nn.Linear(128,10)
        self.dropout = nn.Dropout(p=0.2)
    def forward (self,x):
        x=self.dropout(F.relu(self.fc1(x)))
        x=self.dropout(F.relu(self.fc2(x)))
        x=F.log_softmax(self.fc3(x),dim=1)
        return x
    

model=classifier()
criterion=nn.NLLLoss()
optimizer=optim.Adam(model.parameters(),lr=0.003)
train_on_gpu=torch.cuda.is_available()
if train_on_gpu:
    model.cuda()



**Metrics **
* Cross-validation

In [ ]:
# Cross-validation
epochs=10
train_losses, test_losses = [], []
for e in range(epochs):
    running_loss = 0
    for images, labels in trainloader:
        if train_on_gpu:
            images,labels=images.cuda(),labels.cuda()
        
        optimizer.zero_grad()
        
        log_ps = model(images)
        loss = criterion(log_ps, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
    else:
        test_loss = 0
        accuracy = 0
        
        
        with torch.no_grad():
            model.eval()
            for images, labels in testloader:
                if train_on_gpu:
                    images,labels=images.cuda(),labels.cuda()
                log_ps = model(images)
                test_loss += criterion(log_ps, labels)
                
                ps = torch.exp(log_ps)
                top_p, top_class = ps.topk(1, dim=1)
                equals = top_class == labels.view(*top_class.shape)
                accuracy += torch.mean(equals.type(torch.FloatTensor))
        
        model.train()
        
        train_losses.append(running_loss/len(trainloader))
        test_losses.append(test_loss/len(testloader))

        print("Epoch: {}/{}.. ".format(e+1, epochs),
              "Training Loss: {:.3f}.. ".format(train_losses[-1]),
              "Test Loss: {:.3f}.. ".format(test_losses[-1]),
              "Test Accuracy: {:.3f}".format(accuracy/len(testloader)))
        
plt.plot(train_losses, label='Training set loss')
plt.plot(test_losses, label='Cross-validation set loss')
plt.legend(frameon=False)

In [ ]:
testdata=pd.read_csv('../input/test.csv',dtype=np.float32)
testfeatures = testdata.loc[:].values/255
testvariables = torch.from_numpy(testfeatures)
testlabels=torch.ones_like(testvariables)
testset = torch.utils.data.TensorDataset(testvariables,testlabels)
testloader = torch.utils.data.DataLoader(testset, batch_size = 64, shuffle = False)
ImageId=[]
Label=[]
model.eval()
count=-1
for images,labels in (testloader):
    count+=1
    if train_on_gpu:
        images=images.cuda()
    log_ps = model(images)
    ps = torch.exp(log_ps)
    top_p, top_class = ps.topk(1, dim=1)
    i=0
    for i in range(64):
        try:
            Label.append(top_class[i].item())
            ImageId.append(count*64+i+1)
        except Exception:
            "ok"

**Conclusions **
* Submit results

In [ ]:
# Submit results
result={
    'ImageId':ImageId,
    'Label':Label
}

res = pd.DataFrame(result)

res.to_csv('result.csv',index=False)